<a href="https://colab.research.google.com/github/SiwawawaXD/LLMclassification/blob/main/pythonCallChatgpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from openai import OpenAI
from google.colab import userdata
# Initialize the client
# The client automatically reads the OPENAI_API_KEY environment variable
client = OpenAI(api_key=userdata.get("OPENAI_API_KEY"))

# Upload the training file
file_upload_response = client.files.create(
  file=open("training_data.jsonl", "rb"), # Your file path
  purpose="fine-tune"
)

# Get the file ID
training_file_id = file_upload_response.id
print(f"File uploaded successfully with ID: {training_file_id}")

File uploaded successfully with ID: file-TrTLwMwfN97Jmx8dj1DGyU


In [ ]:
file_upload_response

FileObject(id='file-TrTLwMwfN97Jmx8dj1DGyU', bytes=1885259, created_at=1757065000, filename='training_data.jsonl', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None)

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=userdata.get("OPENAI_API_KEY"))

# The training_file_id from the previous step
training_file_id = "file-TrTLwMwfN97Jmx8dj1DGyU" # <-- Replace with your actual file ID

# Create the fine-tuning job
fine_tuning_job = client.fine_tuning.jobs.create(
  training_file=training_file_id,
  model="gpt-4.1-nano-2025-04-14", # The base model you want to fine-tune
  suffix="Microplastic Classifier 1 trainset" # (Optional) A custom name for your model
)

# Get the job ID
job_id = fine_tuning_job.id
print(f"Fine-tuning job created successfully with ID: {job_id}")

Fine-tuning job created successfully with ID: ftjob-ELoZtofDyFU3gK0k2YlC2IFq


In [ ]:
fine_tuning_job

FineTuningJob(id='ftjob-ELoZtofDyFU3gK0k2YlC2IFq', created_at=1757065012, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-4.1-nano-2025-04-14', object='fine_tuning.job', organization_id='org-XCHqF2z00bEi63AZd5QnYvjt', result_files=[], seed=1319831478, status='validating_files', trained_tokens=None, training_file='file-TrTLwMwfN97Jmx8dj1DGyU', validation_file=None, estimated_finish=None, integrations=[], metadata=None, method=Method(type='supervised', dpo=None, reinforcement=None, supervised=SupervisedMethod(hyperparameters=SupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'))), user_provided_suffix='Microplastic Classifier 1 trainset', usage_metrics=None, shared_with_openai=False, eval_id=None)

In [ ]:
client.fine_tuning.jobs.retrieve(job_id)

FineTuningJob(id='ftjob-ELoZtofDyFU3gK0k2YlC2IFq', created_at=1757065012, error=Error(code='exceeded_quota', message='Creating this fine-tuning job would exceed your hard limit, please check your plan and billing details.                     Cost of job ftjob-ELoZtofDyFU3gK0k2YlC2IFq: USD 6.16. Quota remaining for your project proj_SroCgzfBjKf1IwzrqNv0DVPy: USD 4.68.', param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier=0.1, n_epochs=4), model='gpt-4.1-nano-2025-04-14', object='fine_tuning.job', organization_id='org-XCHqF2z00bEi63AZd5QnYvjt', result_files=[], seed=1319831478, status='failed', trained_tokens=None, training_file='file-TrTLwMwfN97Jmx8dj1DGyU', validation_file=None, estimated_finish=None, integrations=[], metadata=None, method=Method(type='supervised', dpo=None, reinforcement=None, supervised=SupervisedMethod(hyperparameters=SupervisedHyperparameters(batch_size=1, learning_rate_multiplier=0.1, n_epoch

In [ ]:
import numpy as np
from openai import OpenAI
from google.colab import userdata
import pandas as pd
import os
#PVC
folder_path = "/content/Acrylic_SD_50.CSV"
df = pd.read_csv(folder_path, header=None, names=["x", "y"])
xy_pairs = df[["x", "y"]].values.tolist()
step = max(1, len(xy_pairs) // 100)  # keep ~2000 points
xy_pairs = xy_pairs[::step]
print(xy_pairs)

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=userdata.get("OPENAI_API_KEY"))

# Your new custom model ID
fine_tuned_model_id = "ft:gpt-4.1-nano-2025-04-14:personal:microplastic-classifier:CBKMrkFG" # <-- Replace with your new model ID

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are an expert microplastic type classifier(There are Acrylic, Cellulose, ENR, EPDM, HDPE, LDPE, NYLON, PBAT, PBS, PC, PEEK, PEI, PET, PLA, PMMA, POM, PP, PS, PTFE, PU, PVA, PVC) based on the graph points that I send to you. the first column is X-axis and the second column is Y-axis. Return only the unknown label and the microplastic type classname"},
    {"role": "user", "content": f"what is the microplastic type of the graph with the following points? "}
  ]
)

# Print the model's response
print(completion.choices[0].message.content)